<a href="https://colab.research.google.com/github/MayerT1/ACI_Collaboration/blob/main/ACI_cerulean_warbler_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 5.5 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import pandas as pd
from pygbif import occurrences
import geemap.chart as chart

In [3]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


https://www.gbif.org/species/6093350

In [ ]:
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# roi = ee.Geometry.Polygon(coords)

# # Create a map centered on the ROI
# Map = geemap.Map(center=[35.2, -85.63], zoom=11)
# Map.addLayer(roi, {"color": "red"}, "Modeling ROI")
# # Map

In [ ]:
# # Define AOI: Tennessee
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")

# # --------------------------------------------------
# # Fetch Cerulean Warbler occurrences from GBIF
# # --------------------------------------------------
# species_name = "Setophaga cerulea"
# gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)

# # Convert GBIF occurrences to a DataFrame
# records = gbif_data['results']
# df = pd.DataFrame(records)
# df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])

# # Convert to ee.FeatureCollection
# def df_to_fc(row):
#     return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

# presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
# Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# # --------------------------------------------------
# # Generate background (pseudo-absence) points
# # --------------------------------------------------
# background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42)
# background_fc = background_fc.map(lambda f: f.set('occurrence', 0))
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# # Merge presence and background points
# training_data = presence_fc.merge(background_fc)
# Map.addLayer(training_data, {}, "Training Data")

# # --------------------------------------------------
# # Landsat 8 imagery preprocessing
# # --------------------------------------------------
# def maskL8sr(image):
#     qa = image.select('QA_PIXEL')
#     cloudShadowBitMask = 1 << 3
#     cloudsBitMask = 1 << 5
#     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
#         qa.bitwiseAnd(cloudsBitMask).eq(0)
#     )
#     return image.updateMask(mask)

# LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
#         .filterDate('2020-01-01', '2020-12-31') \
#         .filterBounds(roi) \
#         .map(maskL8sr)

# # Compute NDVI median image
# def add_ndvi(image):
#     ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
#     return ndvi.copyProperties(image).set('system:time_start', image.get('system:time_start'))

# image = LS8.map(add_ndvi).median().clip(roi)
# Map.addLayer(image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# # --------------------------------------------------
# # Sample points and train Maxent classifier
# # --------------------------------------------------
# training = image.sampleRegions(
#     collection=training_data,
#     scale=30
# )

# classifier = ee.Classifier.amnhMaxent().train(
#     features=training,
#     classProperty='occurrence',
#     inputProperties=image.bandNames()
# )

# # Classify image
# image_classified = image.classify(classifier)
# # Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

# # Define a gradient from transparent to dark green
# palette = ['#00ffffff', '#006400ff']  # transparent -> dark green

# # Add layer
# Map.addLayer(image_classified.select('probability'),
#              {'min':0, 'max':1, 'palette': palette},
#              'Predicted Probability')



# # Display map
# # Map


Map(center=[35, -86], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [ ]:
# # import ee
# # import geemap
# # import pandas as pd
# # from pygbif import occurrences

# # ee.Initialize()

# # -----------------------------
# # Define AOI: Tennessee
# # -----------------------------
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")

# # # Define your custom ROI
# # coords = [
# #     [-85.85307177259287, 35.06788356661989],
# #     [-85.41087206556162, 35.06788356661989],
# #     [-85.41087206556162, 35.34391870157585],
# #     [-85.85307177259287, 35.34391870157585],
# #     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# # ]

# # # Create EE Geometry and buffer by 10 km (10000 meters)
# # roi = ee.Geometry.Polygon(coords).buffer(100000)


# # -----------------------------
# # GBIF occurrences for Cerulean Warbler
# # -----------------------------
# species_name = "Setophaga cerulea"
# gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)
# records = gbif_data['results']
# df = pd.DataFrame(records).dropna(subset=['decimalLatitude', 'decimalLongitude'])

# def df_to_fc(row):
#     return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

# presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
# Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# # -----------------------------
# # Background points
# # -----------------------------
# background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42).map(lambda f: f.set('occurrence', 0))
# training_data = presence_fc.merge(background_fc)
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# # -----------------------------
# # Landsat 8 NDVI
# # -----------------------------
# def maskL8sr(image):
#     qa = image.select('QA_PIXEL')
#     cloudShadowBitMask = 1 << 3
#     cloudsBitMask = 1 << 5
#     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
#         qa.bitwiseAnd(cloudsBitMask).eq(0)
#     )
#     return image.updateMask(mask)

# LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
#         .filterDate('2020-01-01', '2020-12-31') \
#         .filterBounds(roi) \
#         .map(maskL8sr)

# def add_ndvi(image):
#     return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# ndvi_image = LS8.map(add_ndvi).median().clip(roi)
# Map.addLayer(ndvi_image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# # -----------------------------
# # DEM
# # -----------------------------
# dem = ee.Image("USGS/SRTMGL1_003").clip(roi).rename("DEM")
# Map.addLayer(dem, {'min': 0, 'max': 2000, 'palette': ['white','brown']}, "DEM")

# # -----------------------------
# # Climate: WorldClim v2 bioclimatic variables (1km resolution)
# # -----------------------------
# bio1 = ee.Image("WORLDCLIM/V1/BIO").select('bio01').clip(roi).rename("bio01")  # Mean annual temp
# bio12 = ee.Image("WORLDCLIM/V1/BIO").select('bio12').clip(roi).rename("bio12")  # Annual precipitation

# # -----------------------------
# # Forest canopy ]
# # -----------------------------
# # # TCC and TCH from Global Forest Canopy Height and Cover
# # tcc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").clip(roi).rename("TCC")  # approximate
# # # GEDI products
# tcc = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").first().clip(roi).rename("TCC");

# # -----------------------------
# # Forest canopy / GEDI L2A metrics
# # -----------------------------

# gedi_l2a_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY") \
#                 .filterBounds(roi) \
#                 .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


# gedi_rh98_img = gedi_l2a_fc.reduceToImage(
#     properties=['rh98'],
#     reducer=ee.Reducer.mean()
# ).clip(roi).rename('Rh98')  # no select()


# # gedi_rh98_img = gedi_rh98_img.focal_mean(radius=30, units='meters')


# # -----------------------------
# # Forest canopy / GEDI L2B metrics
# # -----------------------------

# gedi_l2b_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_B_002_MONTHLY") \
#                 .filterBounds(roi) \
#                 .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


# gedi_pai_img = gedi_l2b_fc.reduceToImage(
#     properties=['pai'],
#     reducer=ee.Reducer.mean()
# ).clip(roi).rename('PAI')  # no select()


# # -----------------------------
# # Define square kernel for smoothing
# # -----------------------------
# kernel = ee.Kernel.square(radius=100)  # 100 m like your JS example

# # Apply focal mean directly to single-band images
# gedi_pai_img_smooth = gedi_pai_img.focal_mean(kernel=kernel, iterations=1)
# gedi_rh98_img_smooth = gedi_rh98_img.focal_mean(kernel=kernel, iterations=1)


# # gedi_pai_img = gedi_pai_img.focal_mean(radius=30, units='meters')


# # # List of PAVDZ bands
# # # pai_bands = [f'pai_z{i}' for i in range(15)]
# # pavdz_bands = [f'pavdz_z{i}' for i in range(15)]


# # def rasterize_bands(fc, band_list):
# #     """Convert a list of point-based bands to a raster image stack."""
# #     raster_list = []
# #     for band in band_list:
# #         img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
# #                 .select(['mean']).clip(roi) \
# #                 .rename(band) \
# #                 .focal_mean(radius=30, units='meters')  # smooth gaps
# #         raster_list.append(img)
# #     return ee.Image(raster_list)


# # # Rasterize PAI and PAVDZ
# # # gedi_pai_img = rasterize_bands(gedi_l2b_fc, pai_bands)
# # gedi_pavdz_img = rasterize_bands(gedi_l2b_fc, pavdz_bands)

# # -----------------------------
# # Rasterize GEDI bands with square kernel smoothing
# # -----------------------------

# def rasterize_bands_smooth(fc, band_list, kernel_radius=100):
#     """Convert point-based bands to raster and apply focal mean using a square kernel."""
#     raster_list = []

#     # Define square kernel (like JS example)
#     kernel = ee.Kernel.square(radius=kernel_radius)

#     for band in band_list:
#         img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
#                 .select(['mean']).clip(roi) \
#                 .rename(band)
#         # Apply focal mean using the kernel
#         img_smooth = img.focal_mean(kernel=kernel, iterations=1)
#         raster_list.append(img_smooth)

#     # Combine all smoothed rasters into a single image
#     return ee.Image(raster_list)


# # List of PAVDZ bands
# pavdz_bands = [f'pavdz_z{i}' for i in range(10)]

# # Rasterize and smooth
# gedi_pavdz_img_out = rasterize_bands_smooth(gedi_l2b_fc, pavdz_bands, kernel_radius=100)

# # -----------------------------
# # Combine all predictors into a single image
# # -----------------------------

# predictors = ndvi_image.addBands([dem, bio1, bio12, tcc, gedi_pavdz_img_out])#, gedi_pai_img_smooth, gedi_rh98_img_smooth])
# Map.addLayer(predictors, {}, "All Predictors")

# # -----------------------------
# # Sample points and train Maxent classifier
# # -----------------------------
# training = predictors.sampleRegions(
#     collection=training_data,
#     scale=30
# )

# classifier = ee.Classifier.amnhMaxent().train(
#     features=training,
#     classProperty='occurrence',
#     inputProperties=predictors.bandNames()
# )

# # Classify image
# image_classified = predictors.classify(classifier)
# Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

# Map


wall to wall dev

In [40]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
        !mkdir -p "/content/drive/MyDrive/ACI_Geo_Data"

In [42]:
%cd "/content/drive/MyDrive/ACI_Geo_Data"

/content/drive/MyDrive/ACI_Geo_Data


In [43]:
!pwd

/content/drive/MyDrive/ACI_Geo_Data


In [63]:
# -----------------------------
# Define AOI: Tennessee
# -----------------------------
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")


# -----------------------------
# Smaller ROI around custom point
# -----------------------------
point = ee.Geometry.Point([-85.65568438327509, 35.275624418905814])

# Buffer by 50 km (adjust if needed: 50000 = 50,000 meters)
roi = point.buffer(50000)

# Map setup
Map = geemap.Map(center=[35.275624418905814, -85.65568438327509], zoom=8)
Map.addLayer(roi, {'color': 'red'}, "ROI Small (50 km buffer)")
Map.addLayer(tennessee, {}, "Tennessee AOI")
Map



# # Define your custom ROI
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# # Create EE Geometry and buffer by 10 km (10000 meters)
# roi = ee.Geometry.Polygon(coords).buffer(100000)


# -----------------------------
# GBIF occurrences for Cerulean Warbler
# -----------------------------
species_name = "Setophaga cerulea"
gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)
records = gbif_data['results']
df = pd.DataFrame(records).dropna(subset=['decimalLatitude', 'decimalLongitude'])

def df_to_fc(row):
    return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# -----------------------------
# Background points
# -----------------------------
background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42).map(lambda f: f.set('occurrence', 0))
training_data = presence_fc.merge(background_fc)
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# df

In [64]:
# -----------------------------
# Landsat 8 NDVI
# -----------------------------
def maskL8sr(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate('2020-01-01', '2020-12-31') \
        .filterBounds(roi) \
        .map(maskL8sr)

def add_ndvi(image):
    return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

ndvi_image = LS8.map(add_ndvi).median().clip(roi)
Map.addLayer(ndvi_image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# -----------------------------
# DEM
# -----------------------------
dem = ee.Image("USGS/SRTMGL1_003").clip(roi).rename("DEM")
Map.addLayer(dem, {'min': 0, 'max': 2000, 'palette': ['white','brown']}, "DEM")

# -----------------------------
# Climate: WorldClim v2 bioclimatic variables (1km resolution)
# -----------------------------
bio1 = ee.Image("WORLDCLIM/V1/BIO").select('bio01').clip(roi).rename("bio01")  # Mean annual temp
bio12 = ee.Image("WORLDCLIM/V1/BIO").select('bio12').clip(roi).rename("bio12")  # Annual precipitation
Map.addLayer(bio1, {}, "bio1")
Map.addLayer(bio12, {}, "bio12")
# -----------------------------
# Forest canopy
# -----------------------------
# # TCC and TCH from Global Forest Canopy Height and Cover
# tcc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").clip(roi).rename("TCC")  # approximate
# # GEDI products
tcc = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").first().clip(roi).rename("TCC");
Map.addLayer(tcc, {}, "tcc")
# Map

Map(center=[35.275624418905814, -85.65568438327509], controls=(WidgetControl(options=['position', 'transparent…

In [11]:

# # -----------------------------
# # FM alpha_embeddings
# # -----------------------------
# # Load satellite embeddings for 2020
# alpha_embeddings = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL") \
#                       .filterDate('2020-01-01', '2020-12-31') \
#                       .filterBounds(roi) \
#                       .first() \
#                       .clip(roi)

# # Embeddings come with 128 bands: 'embedding_0' ... 'embedding_127'
# print(alpha_embeddings.bandNames().getInfo())



# def get_pca(image, region, scale=30, n_components=10):
#     # Convert the bands to an array image
#     array_image = image.toArray()

#     # Sample region to compute covariance
#     covar = array_image.reduceRegion(
#         reducer=ee.Reducer.covariance(),
#         geometry=region,
#         scale=scale,
#         maxPixels=1e6
#     )

#     # Extract covariance matrix
#     covar_matrix = ee.Array(covar.get('array'))

#     # Compute eigen decomposition
#     eigens = covar_matrix.eigen()
#     eigen_values = eigens.slice(1, 0, 1)
#     eigen_vectors = eigens.slice(1, 1)

#     # Keep only the top N components
#     eigen_vectors = eigen_vectors.slice(0, 0, n_components)

#     # Turn image into [bands, 1] array
#     array_image2 = image.toArray().toArray(1)

#     # Project the pixels into PCA space
#     pc_image = ee.Image(eigen_vectors) \
#                   .matrixMultiply(array_image2) \
#                   .arrayProject([0]) \
#                   .arrayFlatten(
#                       [['pc' + str(i+1) for i in range(n_components)]]
#                   )
#     return pc_image

# # Run PCA on alpha embeddings (64 bands in your subset)
# alpha_pca = get_pca(alpha_embeddings, region=roi, scale=30, n_components=10)

['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63']


In [67]:
# ============================
# 3. GEDI RH98 data
# ============================
def qualityMask(im):
    return (im.updateMask(im.select('quality_flag').eq(1))
              .updateMask(im.select('degrade_flag').eq(0)))

gedi_l2a = (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
              .map(qualityMask)
              .select('rh98')
              .filterDate('2020-01-01', '2020-12-31')
              .median()
              .clip(roi))

# ============================
# 4. Prepare training data
# ============================
predictor_img = ndvi_image

samples = (predictor_img.addBands(gedi_l2a)
            .sample(region=roi,
                    scale=30,
                    projection=predictor_img.projection(),
                    geometries=True)
            .filter(ee.Filter.notNull(['rh98'])))

# ============================
# 5. Train RF regression
# ============================
rf_rh98 = (ee.Classifier.smileRandomForest(numberOfTrees=50)
              .setOutputMode('REGRESSION')
              .train(features=samples,
                     classProperty='rh98',
                     inputProperties=predictor_img.bandNames()))

# ============================
# 6. Predict RH98
# ============================
rh98_prediction = predictor_img.classify(rf_rh98).rename('rh98_pred')

# ============================
# 7. Visualization
# ============================
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['white','green']}
gedi_vis = {'min': 1, 'max': 60, 'palette': ['darkred','red','orange','green','darkgreen']}
pred_vis = {'min': 0, 'max': 40, 'palette': ['white','lightblue','blue','darkblue']}

Map.centerObject(roi, 10)
Map.addLayer(ndvi_image, ndvi_vis, 'NDVI')
Map.addLayer(gedi_l2a, gedi_vis, 'GEDI RH98')
Map.addLayer(rh98_prediction, pred_vis, 'RF RH98 Prediction (NDVI)')
Map.addLayer(roi, {}, 'ROI')

# Map

Map(bottom=26197.0, center=[35.275675711257, -85.6556826409482], controls=(WidgetControl(options=['position', …

In [69]:
# ============================
# 1. GEDI L2B PAI data
# ============================
def qualityMask(im):
    return (im.updateMask(im.select('l2b_quality_flag').eq(1))
              .updateMask(im.select('degrade_flag').eq(0)))

gedi_l2b = (ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')
              .map(qualityMask)
              .select('pai')
              .filterDate('2020-01-01', '2020-12-31')
              .median()
              .clip(roi))

# Visualization
pai_vis = {'min': 1, 'max': 60,
           'palette': ['darkred','red','orange','green','darkgreen']}
Map.addLayer(gedi_l2b, pai_vis, 'GEDI L2B PAI')

print('GEDI L2B PAI image:', gedi_l2b.getInfo())

# ============================
# 2. Prepare training data
# ============================
predictor_img = ndvi_image  # single predictor: NDVI

samples_b = (predictor_img.addBands(gedi_l2b)
              .sample(region=roi,
                      scale=30,
                      projection=predictor_img.projection(),
                      geometries=True)
              .filter(ee.Filter.notNull(['pai'])))

# ============================
# 3. Train RF regression
# ============================
rf_pai = (ee.Classifier.smileRandomForest(numberOfTrees=50)
            .setOutputMode('REGRESSION')
            .train(features=samples_b,
                   classProperty='pai',
                   inputProperties=predictor_img.bandNames()))

# ============================
# 4. Predict PAI
# ============================
rf_pai_prediction = predictor_img.classify(rf_pai).rename('pai_pred')

# ============================
# 5. Visualization
# ============================
pai_pred_vis = {'min': 0, 'max': 10,
                'palette': ['white','lightblue','blue','darkblue']}

Map.centerObject(roi, 10)
Map.addLayer(rf_pai_prediction, pai_pred_vis, 'RF PAI Prediction (NDVI)')

# Map

GEDI L2B PAI image: {'type': 'Image', 'bands': [{'id': 'pai', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3, 3], 'origin': [-87, 34], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-85.65568438327509, 35.72556771647989], [-85.81045412939606, 35.707561598626754], [-85.95270944453374, 35.654999793901034], [-86.07099426598994, 35.57212852555563], [-86.1558646071102, 35.465625735472834], [-86.20064834502853, 35.344043671708334], [-86.20194869799064, 35.21710476279884], [-86.15986100165567, 35.09491416795262], [-86.07790304990861, 34.98715567368491], [-85.96268390608564, 34.90233450435306], [-85.82335273729818, 34.84712282956825], [-85.6708783626476, 34.82585307945494], [-85.5172138519597, 34.840192062366924], [-85.37440113624679, 34.88901623606711], [-85.25367027761644, 34.96849574410549], [-85.16458799629929, 35.07238209969832], [-85.11431037629185, 35.192481660616934], [-85.

Map(bottom=103889.0, center=[35.275675711257, -85.6556826409482], controls=(WidgetControl(options=['position',…

In [78]:
# ============================
# Loop through PAI_z bands
# ============================

# List of bands: pai_z0 ... pai_z30
pai_bands = [f"pai_z{i}" for i in range(31)]

# Dictionary to hold models and predictions
rf_models = {}
rf_predictions = {}

for band in pai_bands:
    # Load GEDI L2B band
    gedi_band = (ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')
                   .map(qualityMask)
                   .select(band)
                   .filterDate('2020-01-01', '2020-12-31')
                   .median()
                   .clip(roi))

    # Sample NDVI + GEDI band
    samples = (ndvi_image.addBands(gedi_band)
                .sample(region=roi,
                        scale=30,
                        projection=ndvi_image.projection(),
                        geometries=True)
                .filter(ee.Filter.notNull([band])))

    # Train RF regressor
    rf_model = (ee.Classifier.smileRandomForest(numberOfTrees=50)
                  .setOutputMode('REGRESSION')
                  .train(features=samples,
                         classProperty=band,
                         inputProperties=ndvi_image.bandNames()))

    # Predict band from NDVI
    rf_pred = ndvi_image.classify(rf_model).rename(f"{band}_pred")

    # Save results
    rf_models[band] = rf_model
    rf_predictions[band] = rf_pred

print("✅ Trained and predicted for all bands: pai_z0 ... pai_z30")


✅ Trained and predicted for all bands: pavd_z0 ... pavd_z30


In [86]:
# ============================
# Loop through PAVD_z bands
# ============================

# Correct list of bands: pavd_z0 ... pavd_z30
pavd_bands = [f"pavd_z{i}" for i in range(29)]

# Dictionaries to hold models and predictions
rf_models_pavd = {}
rf_predictions_pavd = {}

# Inspect first image to check available bands
first_img = ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY').first()
available_bands = first_img.bandNames().getInfo()

for band in pavd_bands:
    # Skip bands that are not in the collection
    if band not in available_bands:
        print(f"⚠️ Band {band} not available, skipping.")
        continue

    # Load GEDI L2B band
    gedi_band = (ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')
                   .map(qualityMask)
                   .select(band)
                   .filterDate('2020-01-01', '2020-12-31')
                   .median()
                   .clip(roi))

    # Sample NDVI + GEDI band
    samples = (ndvi_image.addBands(gedi_band)
                .sample(region=roi,
                        scale=30,
                        projection=ndvi_image.projection(),
                        geometries=True)
                .filter(ee.Filter.notNull([band])))

    # Train RF regressor
    rf_model = (ee.Classifier.smileRandomForest(numberOfTrees=50)
                  .setOutputMode('REGRESSION')
                  .train(features=samples,
                         classProperty=band,
                         inputProperties=ndvi_image.bandNames()))

    # Predict band from NDVI
    rf_pred = ndvi_image.classify(rf_model).rename(f"{band}_pred")

    # Save results
    rf_models_pavd[band] = rf_model
    rf_predictions_pavd[band] = rf_pred

print("✅ Trained and predicted for all available pavd_z bands")


✅ Trained and predicted for all available pavd_z bands


In [81]:
# Access a specific predicted band
pred_z5 = rf_predictions_pavd['pavd_z5']  # ee.Image

print(pred_z5.getInfo())

{'type': 'Image', 'bands': [{'id': 'pavd_z5_pred', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3, 3], 'origin': [-87, 34], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-85.65568438327509, 35.72556771647989], [-85.81045412939606, 35.707561598626754], [-85.95270944453374, 35.654999793901034], [-86.07099426598994, 35.57212852555563], [-86.1558646071102, 35.465625735472834], [-86.20064834502853, 35.344043671708334], [-86.20194869799064, 35.21710476279884], [-86.15986100165567, 35.09491416795262], [-86.07790304990861, 34.98715567368491], [-85.96268390608564, 34.90233450435306], [-85.82335273729818, 34.84712282956825], [-85.6708783626476, 34.82585307945494], [-85.5172138519597, 34.840192062366924], [-85.37440113624679, 34.88901623606711], [-85.25367027761644, 34.96849574410549], [-85.16458799629929, 35.07238209969832], [-85.11431037629185, 35.192481660616934], [-85.106994371562

In [82]:
Map.addLayer(pred_z5, {'min': 0, 'max': 10, 'palette': ['white','yellow','green','darkgreen']}, 'pai_z5_pred')
# Map

Map(bottom=207478.0, center=[35.27589521097769, -85.6549072265625], controls=(WidgetControl(options=['position…

In [76]:
predictors = ee.Image.cat([ndvi, dem, bio1, bio12, tcc, rh98_prediction, rf_pai_prediction]).resample('bilinear').reproject(proj)
Map.addLayer(predictors, {}, "predictors")
# Map

# predictors = predictors.addBands(align(gedi_predictions))

Map(bottom=207384.0, center=[35.27589521097769, -85.6549072265625], controls=(WidgetControl(options=['position…

In [87]:
# Loop through RF PAVD models and add their predictions to the predictor stack
for band_name, rf_model in rf_models_pavd.items():
    # Predict the PAVD band from NDVI
    predicted_band = ndvi.classify(rf_model).rename(band_name + '_pred')

    # Add the predicted band to the existing predictor stack
    predictors = predictors.addBands(predicted_band)

# Optional: check all band names
print("Updated predictor stack bands:", predictors.bandNames().getInfo())


EEException: reduce.median: Error in map(ID=202001_000E_000N):
Image.select: Band pattern 'pai_z30' did not match any bands. Available bands: [algorithmrun_flag, beam, cover, cover_z0, cover_z1, cover_z2, cover_z3, cover_z4, cover_z5, cover_z6, cover_z7, cover_z8, cover_z9, cover_z10, cover_z11, cover_z12, cover_z13, cover_z14, cover_z15, cover_z16, cover_z17, cover_z18, cover_z19, cover_z20, cover_z21, cover_z22, cover_z23, cover_z24, cover_z25, cover_z26, cover_z27, cover_z28, cover_z29, degrade_flag, delta_time, fhd_normal, l2b_quality_flag, local_beam_azimuth, local_beam_elevation, pai, pai_z0, pai_z1, pai_z2, pai_z3, pai_z4, pai_z5, pai_z6, pai_z7, pai_z8, pai_z9, pai_z10, pai_z11, pai_z12, pai_z13, pai_z14, pai_z15, pai_z16, pai_z17, pai_z18, pai_z19, pai_z20, pai_z21, pai_z22, pai_z23, pai_z24, pai_z25, pai_z26, pai_z27, pai_z28, pai_z29, pavd_z0, pavd_z1, pavd_z2, pavd_z3, pavd_z4, pavd_z5, pavd_z6, pavd_z7, pavd_z8, pavd_z9, pavd_z10, pavd_z11, pavd_z12, pavd_z13, pavd_z14, pa

In [ ]:


# # Make sure your ROI is defined
# roi = tennessee.geometry()  # or whatever ROI you are using

# # Export PAI
# task_pai = ee.batch.Export.image.toDrive(
#     image=pai_prediction,
#     description='GEDI_PAI_RF',
#     folder='/content/drive/MyDrive/ACI_Geo_Data',
#     fileNamePrefix='gedi_pai_rf',
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task_pai.start()

# # Export RH98
# task_rh98 = ee.batch.Export.image.toDrive(
#     image=rh98_prediction,
#     description='GEDI_RH98_RF',
#     folder='/content/drive/MyDrive/ACI_Geo_Data',
#     fileNamePrefix='gedi_rh98_rf',
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task_rh98.start()

# # Pavdz bands
# for band in pavdz_bands:
#     img = pavdz_predictions.select(band)
#     task = ee.batch.Export.image.toDrive(
#         image=img,
#         description=f'GEDI_{band}_RF',
#         folder='/content/drive/MyDrive/ACI_Geo_Data',
#         fileNamePrefix=f'gedi_{band}_rf',
#         region=roi,
#         scale=30,
#         maxPixels=1e13
#     )
#     task.start()


In [ ]:
# import rasterio
# import os

# drive_path = '/content/drive/MyDrive/ACI_Geo_Data/'

# pai_tif = os.path.join(drive_path, 'gedi_pai_rf.tif')
# rh98_tif = os.path.join(drive_path, 'gedi_rh98_rf.tif')
# pavdz_tifs = [os.path.join(drive_path, f'gedi_{b}_rf.tif') for b in pavdz_bands]

# # Open as rasterio objects
# pai_img = rasterio.open(pai_tif)
# rh98_img = rasterio.open(rh98_tif)
# pavdz_imgs = [rasterio.open(tif) for tif in pavdz_tifs]


In [38]:
# gedi_predictions = pavdz_predictions.addBands([pai_prediction, rh98_prediction])
# predictors = predictor_stack.addBands([gedi_predictions])

# # Map.addLayer(gedi_predictions, {}, "RF GEDI Predictions")


In [39]:
# # -----------------------------
# # Predictor stack
# # -----------------------------
# proj = ndvi.projection()
# def align(img): return img.resample('bilinear').reproject(proj)

# predictors = ee.Image.cat([ndvi, dem, bio1, bio12, tcc]).resample('bilinear').reproject(proj)
# predictors = predictors.addBands(align(gedi_predictions))

# -----------------------------
# Train MaxEnt
# -----------------------------
training = predictors.sampleRegions(collection=training_data, scale=30)
classifier = ee.Classifier.amnhMaxent().train(training, 'occurrence', predictors.bandNames())
image_classified = predictors.classify(classifier)

# -----------------------------
# Visualization (small ROI)
# -----------------------------
Map.addLayer(image_classified.clip(roi_small).select('probability'),
             {'min':0,'max':1,'palette':['white','lightgreen','green']},
             'Predicted Probability (ROI sample)')

# -----------------------------
# Export full Tennessee prediction
# -----------------------------
task = ee.batch.Export.image.toDrive(
    image=image_classified.select('probability'),
    description='cerulean_warbler_maxent_prob',
    folder='EarthEngine',
    fileNamePrefix='cerulean_warbler_maxent',
    region=roi,
    scale=30,
    maxPixels=1e13
)
task.start()
print("Export started → Check Earth Engine Tasks tab or Drive for results.")

Map

EEException: Computation timed out.

In [ ]:
# training = predictors.sampleRegions(
#     collection=training_data,
#     scale=30
# )

# classifier = ee.Classifier.amnhMaxent().train(
#     features=training,
#     classProperty='occurrence',
#     inputProperties=predictors.bandNames()
# )

# image_classified = predictors.classify(classifier)
# Map.addLayer(image_classified.select('probability'),
#              {'min':0, 'max':1, 'palette': ['white','lightgreen','green']},
#              'Predicted Probability')
# Map